In [1]:
import os
import sys
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_theme()

In [2]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
from src.strategy.model import Model
from src.strategy.environment import Environment
from src.strategy.agent import Agent
from src.strategy.buffer import Buffer
from src.utils import get_config, read_file
config = get_config.read_yaml()

In [4]:
print("Starting Training...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Starting Training...
Using device: cuda


In [5]:
MODEL_PATH = config['paths']['model_directory']

hp = config['hyperparameters']
NUM_ASSETS = hp['num_assets']
INPUT_DIM = hp['input_dim']
ACTION_DIM = hp['action_dim']
NUM_LSTM_LAYERS = hp['num_lstm_layers']
HIDDEN_STATE_DIM = hp['hidden_state_dim']
ACTOR_HIDDEN_DIM = hp['actor_hidden_dim']
CRITIC_HIDDEN_DIM = hp['critic_hidden_dim']
GAMMA = hp['gamma']
GAE_LAMBDA = hp['gae_lambda']
CLIP_EPSILON = hp['clip_epsilon']
VALUE_LOSS_COEF = hp['value_loss_coef']
ENTROPY_LOSS_COEF = hp['entropy_loss_coef']
LEARNING_RATE = hp['learning_rate']
NUM_EPOCHS = hp['num_epochs']
ROLLOUT_SIZE = hp['rollout_size']
MINI_BATCH_SIZE = hp['mini_batch_size']
SEQUENCE_LENGTH = hp['seq_len']
train_data_norm = read_file.read_merged_training_data()
train_data_unnorm = read_file.read_merged_training_data(False)
print(f'MODEL_PATH: {MODEL_PATH}')
print(f'NUM_ASSETS: {NUM_ASSETS}')
print(f'INPUT_DIM: {INPUT_DIM}')
print(f'ACTION_DIM: {ACTION_DIM}')
print(f'NUM_LSTM_LAYERS: {NUM_LSTM_LAYERS}')
print(f'HIDDEN_STATE_DIM: {HIDDEN_STATE_DIM}')
print(f'ACTOR_HIDDEN_DIM: {ACTOR_HIDDEN_DIM}')
print(f'CRITIC_HIDDEN_DIM: {CRITIC_HIDDEN_DIM}')
print(f'GAMMA: {GAMMA}')
print(f'GAE_LAMBDA: {GAE_LAMBDA}')
print(f'CLIP_EPSILON: {CLIP_EPSILON}')
print(f'VALUE_LOSS_COEF: {VALUE_LOSS_COEF}')
print(f'ENTROPY_LOSS_COEF: {ENTROPY_LOSS_COEF}')
print(f'LEARNING_RATE: {LEARNING_RATE}')
print(f'NUM_EPOCHS: {NUM_EPOCHS}')
print(f'ROLLOUT_SIZE: {ROLLOUT_SIZE}')
print(f'BATCH_SIZE: {MINI_BATCH_SIZE}')
print(f'SEQUENCE_LENGTH: {SEQUENCE_LENGTH}')
print(train_data_norm)
print(train_data_unnorm)

MODEL_PATH: models/
NUM_ASSETS: 10
INPUT_DIM: 126
ACTION_DIM: 10
NUM_LSTM_LAYERS: 1
HIDDEN_STATE_DIM: 256
ACTOR_HIDDEN_DIM: 64
CRITIC_HIDDEN_DIM: 64
GAMMA: 0.99
GAE_LAMBDA: 0.95
CLIP_EPSILON: 0.2
VALUE_LOSS_COEF: 0.5
ENTROPY_LOSS_COEF: 0.01
LEARNING_RATE: 0.001
NUM_EPOCHS: 10
ROLLOUT_SIZE: 4096
BATCH_SIZE: 128
SEQUENCE_LENGTH: 72
                     ('open', 'ETH')  ('high', 'ETH')  ('low', 'ETH')  \
timestamp                                                               
2020-10-01 00:00:00         1.000000         0.962171        0.986505   
2020-10-01 01:00:00         0.922741         1.000000        0.971660   
2020-10-01 02:00:00         1.000000         1.000000        1.000000   
2020-10-01 03:00:00         1.000000         0.932530        1.000000   
2020-10-01 04:00:00         0.970954         1.000000        1.000000   
...                              ...              ...             ...   
2024-09-29 19:00:00         0.616359         0.535242        0.735184   
2024-09-29 

In [6]:
model = Model(INPUT_DIM,
              HIDDEN_STATE_DIM,
              NUM_ASSETS,
              NUM_LSTM_LAYERS,
              ACTOR_HIDDEN_DIM,
              CRITIC_HIDDEN_DIM,)
model

Model(
  (lstm): LSTM(126, 256, batch_first=True)
  (actor_head): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=20, bias=True)
  )
  (critic_head): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [7]:
agent = Agent(model,
              GAMMA,
              GAE_LAMBDA,
              CLIP_EPSILON,
              VALUE_LOSS_COEF,
              ENTROPY_LOSS_COEF,
              LEARNING_RATE,
              MINI_BATCH_SIZE,
              device,
              MODEL_PATH)
agent

In [8]:
env = Environment(train_data_unnorm,
                  SEQUENCE_LENGTH,
                  NUM_ASSETS)
env

In [9]:
env.reset(0)

In [10]:
buffer = Buffer(ROLLOUT_SIZE,
                SEQUENCE_LENGTH,
                INPUT_DIM,
                ACTION_DIM,
                device)
buffer.display()

--- Buffer Contents (all on CPU) ---
states: torch.Size([4096, 72, 126])
actions: torch.Size([4096, 10])
log_probs: torch.Size([4096])
values: torch.Size([4096])
rewards: torch.Size([4096])
dones: torch.Size([4096])
advantages: torch.Size([4096])
returns: torch.Size([4096])
current_step: 0 / 4096
